# Create Biological Intuition From a Model

In response to reviewer concerns, we use data simulated data to create biological intuition.  In this case we take a set of data and generate a dynamic model.  We then use that dynamic model to simulate a held back set of strains to see final production.  Final production is overlayed onto a PCA of the proteomics at the Final Time. In this way we can understand the proteomics profile that improves production purely from simulation.

In [1]:
%matplotlib inline
from KineticLearning import learn_dynamics,read_timeseries_data,simulate_dynamics

#Visualization Tools
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA

from IPython.display import display
import numpy as np

## Reading Data & Preparing the DataFrame

Data is read in using the `read_timeseries_data` function. This also prepares it for processing by doing data imputation, data augmentation, and derivative estimation on the time series.  In this case we are reading in a large data set so the number of strains that are read is limited to 200.  This speeds up the import process dramatically. 

In [ ]:
#Read In Raw DataFrame & Put it into Apropriate Format
proteins = ['AtoB', 'HMGS', 'HMGR', 'MK', 'PMK', 'PMD', 'Idi','GPPS', 'LS']
metabolites = ['GPPS', 'LS', 'Acetyl-CoA', 'Acetoacetyl-CoA', 'HMG-CoA', 'Mev', 'MevP','MevPP', 'IPP', 'DMAPP', 'GPP', 'Limonene']

In [ ]:
#Pull Data from Simulated Data Set
df = read_timeseries_data('data/Fulld10000n0.csv',metabolites,proteins,augment=None,n=100)

states                                                     \
                     GPPS            LS Acetyl-CoA Acetoacetyl-CoA   HMG-CoA   
Strain Time                                                                    
1      0     8.546974e+00  7.102368e+00   0.200000        0.200000  0.200000   
       1     3.488546e+06  6.205517e+06   0.202057        0.200050  0.198739   
       2     3.857668e+06  7.334069e+06   0.204079        0.200136  0.197191   
       3     3.998702e+06  7.807357e+06   0.206089        0.200233  0.195611   
       4     4.073159e+06  8.067673e+06   0.208094        0.200337  0.194026   
       5     4.119178e+06  8.232364e+06   0.210094        0.200444  0.192443   
       6     4.150440e+06  8.345946e+06   0.212090        0.200555  0.190867   
       7     4.173062e+06  8.429013e+06   0.214085        0.200668  0.189299   
       8     4.190191e+06  8.492407e+06   0.216077        0.200784  0.187740   
       9     4.203611e+06  8.542377e+06   0.218068        0.200901  0.186193   
       10    4.214409e+06  8.582778e+06   0.220056        0.201020  0.184656   
       11    4.223286e+06  8.616118e+06   0.222043        0.201140  0.183130   
       12    4.230711e+06  8.644101e+06   0.224029        0.201262  0.181615   
       13    4.237014e+06  8.667921e+06   0.226013        0.201386  0.180113   
       14    4.242432e+06  8.688442e+06   0.227996        0.201510  0.178621   
       15    4.247139e+06  8.706307e+06   0.229977        0.201636  0.177142   
       16    4.251266e+06  8.721998e+06   0.231958        0.201764  0.175674   
       17    4.254914e+06  8.735891e+06   0.233937        0.201892  0.174218   
       18    4.258162e+06  8.748277e+06   0.235915        0.202022  0.172773   
       19    4.261073e+06  8.759389e+06   0.237891        0.202153  0.171341   
       20    4.263695e+06  8.769414e+06   0.239867        0.202285  0.169920   
       21    4.266071e+06  8.778504e+06   0.241841        0.202418  0.168510   
       22    4.268233e+06  8.786785e+06   0.243814        0.202553  0.167112   
       23    4.270209e+06  8.794358e+06   0.245787        0.202688  0.165726   
       24    4.272022e+06  8.801312e+06   0.247758        0.202825  0.164351   
       25    4.273691e+06  8.807720e+06   0.249728        0.202962  0.162987   
       26    4.275233e+06  8.813643e+06   0.251696        0.203101  0.161635   
       27    4.276662e+06  8.819134e+06   0.253664        0.203241  0.160293   
       28    4.277990e+06  8.824239e+06   0.255631        0.203381  0.158964   
       29    4.279226e+06  8.828998e+06   0.257597        0.203523  0.157645   
...                   ...           ...        ...             ...       ...   
10000  40    6.705666e+06  1.007370e+06   0.276744        0.207560  0.166500   
       41    6.706647e+06  1.007499e+06   0.278626        0.207785  0.165691   
       42    6.707581e+06  1.007622e+06   0.280508        0.208011  0.164887   
       43    6.708472e+06  1.007739e+06   0.282388        0.208239  0.164087   
       44    6.709322e+06  1.007851e+06   0.284266        0.208468  0.163290   
       45    6.710135e+06  1.007957e+06   0.286144        0.208698  0.162498   
       46    6.710913e+06  1.008060e+06   0.288019        0.208930  0.161709   
       47    6.711658e+06  1.008157e+06   0.289894        0.209163  0.160924   
       48    6.712372e+06  1.008251e+06   0.291767        0.209397  0.160143   
       49    6.713057e+06  1.008341e+06   0.293639        0.209633  0.159366   
       50    6.713714e+06  1.008428e+06   0.295509        0.209870  0.158593   
       51    6.714346e+06  1.008511e+06   0.297379        0.210109  0.157823   
       52    6.714954e+06  1.008590e+06   0.299246        0.210348  0.157058   
       53    6.715539e+06  1.008667e+06   0.301113        0.210589  0.156296   
       54    6.716103e+06  1.008741e+06   0.302978        0.210832  0.155538   
       55    6.716646e+06  1.008813e+06   0.304842        0.211076  0.154784   
       56    6.7171

/Users/zakcostello/anaconda/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


Once the Data is read in,  it is split into test and training groups for building the model and using it for prediction.  100 strains go into each group.  Then the training dataframe is used to train the model. Parameters are set to reduce the training time to a manageable duration.

In [ ]:
#Sample 100 Strains for Training & 100 Strains For Prediction
strains = df.index.get_level_values(0).unique()
test_df = df.loc[df.index.get_level_values(0).isin(strains[0:30])]
training_df = df.loc[df.index.get_level_values(0).isin(strains[50:])]

model = learn_dynamics(training_df,generations=10,population_size=10)

Once the Model is constructed, we use the model to predict the metabolite trajectory on the 100 selected test strains. 

In [ ]:
# Use Model to Metabolite Trajectory for Each Strain
trajectory_df = test_df.groupby('Strain').apply(lambda x: simulate_dynamics(model,x,tolerance=5e-4))

The Predicted vs Actual Final Productions are pulled from the data and the error distribution is plotted.

In [ ]:
#Find the Final Production in each Trajectory for both the real
display(trajectory_df)

y_p = trajectory_df['Limonene'].loc[trajectory_df.index.get_level_values(1)==69]
y = test_df['states']['Limonene'].loc[trajectory_df.index.get_level_values(1)==69]
y_err = y-y_p

sns.distplot(y_err)
plt.show()

Finally We Plot the PCA with production predictions overlayed onto the data to demonstrate that there is a relationship that can be determined between the PCA and production. 

In [ ]:
#Plot PCA overlayed onto Predictions
X = test_df['controls'].loc[trajectory_df.index.get_level_values(1)==69]
pca = PCA(2)
X_pca = np.transpose(pca.fit_transform(X)).tolist()

#Scatter Plot PCA Fit
plt.figure()
plt.scatter(*X_pca,c=y*1000,cmap=plt.cm.get_cmap('coolwarm'))
plt.colorbar()
plt.title('Proteomics data overlayed with Actual Final Production')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

plt.figure()
plt.scatter(*X_pca,c=y_p*1000,cmap=plt.cm.get_cmap('coolwarm'))
plt.colorbar()
plt.title('Proteomics data overlayed with Predicted Final Production')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [ ]:
display(trajectory_df)
trajectory_df.to_csv('FullTimeSeriesPredicted.csv')

In [ ]:
import pandas as pd
pd.concat([X,y,y_p],axis=1).to_csv('Predicted_Data2.csv')

#display(X)
#display(y)
#display(y_p)